**Goal**: Explore Transformers library by HuggingFace  for deep learning model creation and Sentiment Analysis

**Dataset**:https://www.kaggle.com/nicapotato/womens-ecommerce-clothing-reviews

In [0]:
import pandas as pd
import numpy as np
import torch
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="1"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [0]:
dataset = pd.read_csv("/content/drive/My Drive/pytorch_tutorials/PyTorch Sentiment Analysis/data/Womens Clothing E-Commerce Reviews.csv")

In [7]:
dataset.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


As we are concerned about only text data analysis,we'll ignore all other columns and keep 'Review Text' and 'Recommended IND' columns only


In [0]:
dataset = dataset[['Review Text','Recommended IND']]

In [0]:
dataset =  dataset.rename(columns={'Review Text':'review','Recommended IND':'recommended'})

In [10]:
dataset.head() 

,review,recommended
0,Absolutely wonderful - silky and sexy and comf...,1
1,Love this dress! it's sooo pretty. i happene...,1
2,I had such high hopes for this dress and reall...,0
3,"I love, love, love this jumpsuit. it's fun, fl...",1
4,This shirt is very flattering to all due to th...,1


In [11]:
dataset.shape

(23486, 2)

In [12]:
dataset.review.isnull().sum()

845

In [0]:
dataset = dataset.dropna(axis=0, subset=['review'])

In [14]:
dataset.recommended.value_counts()

1    18540
0     4101
Name: recommended, dtype: int64

In [0]:
'''Since dataset is huge to run on Google colab,let's take only few samples for our tutorial'''
dataset = dataset.iloc[:2000,:]


To learn about BERT and different language models:<br>
http://jalammar.github.io/illustrated-transformer/ <br>
http://jalammar.github.io/illustrated-bert/<br>
This blog is the best explaination you could find on the internet.<br>

Transformers library by HuggingFace provides many pretrained language models which can be further used/fine tuned to specific NLP tasks.<br>
More info: https://github.com/huggingface/transformers





Now we'll do sentiment analysis/senetence classification in following 2 steps:
1. Load Pretrained DistilBert model architecture and fine-tune it further using logistic regression<br>
2. Load Pretrained DistilBert classification class and fine-tune it further using PyTorch <br>

Difference between 1.DistilBert model architecure and 2.DistilBert classification class:<br>
DistilBert model architecure provides last hidden states from the model.These last hidden states can be used by any classification model further.For example we are going to use them in logistic regression.<br>
DistilBert classification class uses these last hidden state and initialises weights for classification task.These weights need to be trained/fine-tuned further.

**1. Load Pretrained DistilBert model architecture**

Let's first install transformers library

In [16]:
!pip install transformers

     |████████████████████████████████| 573kB 3.3MB/s 
     |████████████████████████████████| 890kB 41.7MB/s 
     |████████████████████████████████| 1.0MB 41.7MB/s 
     |████████████████████████████████| 3.7MB 33.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=4d59110ace37f8794c8a585890549f631dfcb9270a5e8dd31b0d78e09decaf2a
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
from transformers import DistilBertModel,DistilBertTokenizer

In [18]:
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [0]:
#Tokenization:Convert words in 'review' column to numbers
tokenized_reviews = dataset.review.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

In [20]:
#Padding:To make all sentences of same length.This is only required for batch creation
'''First we need to find maximum length of senetence/review.'''
max_len = max(map(len,tokenized_reviews))
print(max_len)

148


In [21]:
np.array(tokenized_reviews).shape

(2000,)

In [0]:
padded_reviews = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_reviews])

In [23]:
np.array(padded_reviews).shape

(2000, 148)

In [24]:
padded_reviews[0]

array([  101,  7078,  6919,  1011, 18848,  1998,  7916,  1998,  6625,
         102,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [0]:
#Masking:To tell DistilBert to ignore padding. This is called attention masking.Basically it'll put 1 where encoding is present and 0 for others.
attention_masked_reviews = np.where(padded_reviews!=0,1,0)

In [26]:
np.array(attention_masked_reviews).shape

(2000, 148)

In [27]:
attention_masked_reviews[0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
# Get last hidden states
input_ids = torch.tensor(padded_reviews).to(device)
attention_mask = torch.tensor(attention_masked_reviews).to(device)

with torch.no_grad():
  last_hidden_states = model(input_ids,attention_mask=attention_mask)

But we don't need all hidden states.We only need last hidden state of first token 'CLS' of each sentence. This is added by BERT for classification purpose.<br>


In [29]:
print(type(last_hidden_states))

<class 'tuple'>


In [30]:
last_hidden_states[0].shape

torch.Size([2000, 148, 768])

In [0]:
'''Dimension of hidden state axbxc
a = number of reviews  2000
b = number of tokens  148
c = number of hidden units  768 '''
X = last_hidden_states[0][:,0,:].numpy()
y = dataset.recommended

In [32]:
print(X.shape)
print(y.shape)

(2000, 768)
(2000,)


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#Logistic Regression
X_train,X_test,y_train,y_test = train_test_split(X,y)
log_model = LogisticRegression(max_iter=1500)
log_model.fit(X_train,y_train)
preds = log_model.predict(X_test)
from sklearn import metrics
print(metrics.roc_auc_score(y_test, preds))

0.7583053910727451


**2.Load Pretrained DistilBert classification class**

In [0]:
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


In [0]:
tokenized_reviews = dataset.review.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))
max_len = max(map(len,tokenized_reviews))
padded_reviews = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_reviews])
attention_masked_reviews = np.where(padded_reviews!=0,1,0)



In [0]:
#Dataset preparation
from torch.utils.data import Dataset, TensorDataset,DataLoader
from sklearn.model_selection import train_test_split

X = torch.tensor(padded_reviews)
X_attention = torch.tensor(attention_masked_reviews)
#y = torch.tensor(np.array(dataset.recommended.values))
y = torch.tensor(np.array(dataset.recommended.values)[:,np.newaxis], dtype=torch.float32)

X_train,X_test,y_train,y_test = X[:1500],X[1500:],y[:1500],y[1500:]
X_train_attention,X_test_attention = X_attention[:1500],X_attention[1500:]

train_data = TensorDataset(X_train, X_train_attention, y_train)

train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

In [55]:
y_train.shape

torch.Size([1500, 1])

In [0]:
#Model training
NUM_EPOCHS = 1
LEARNING_RATE = 0.01
optimizer =torch.optim.SGD(model.parameters(), lr=LEARNING_RATE) 
loss_fn = torch.nn.BCEWithLogitsLoss()

for i in range(NUM_EPOCHS):
  model.train()
  for X_batch,X_attention_batch,y_batch in train_loader:
    output = model(X_batch,attention_mask=X_attention_batch,labels=None)
    y_pred = output[0]
    #print(y_pred)
    loss = loss_fn(y_pred,y_batch)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


In [61]:
#Evaluation
test_dataset = TensorDataset(X_test, X_test_attention)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

preds = np.zeros([len(test_dataset), 1])
model.eval()
for i, (x_batch, x_mask) in enumerate(test_loader):
    outputs = model(x_batch.to(device),
                    attention_mask=x_mask.to(device))
    y_pred = sigmoid(outputs[0].detach().cpu().numpy())
    preds[i*16:(i+1)*16, :] = y_pred
    
from sklearn import metrics
print(metrics.roc_auc_score(y_test, preds))

0.8644878078114195
